<a href="https://colab.research.google.com/github/WenqiLiao/Underground_Utilities/blob/main/FormattedAngleCostFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.ops import linemerge
from shapely.wkt import loads
from shapely.geometry import LineString
from shapely.geometry import Point, Polygon
from scipy.spatial import Delaunay

In [ ]:
df_point = gpd.read_file('/content/hydrant_location.shp')
df_point['id'] = df_point.reset_index().index
df_point

,id,geometry
0,0,POINT (-74.00698 40.74748)
1,1,POINT (-74.00722 40.74722)
2,2,POINT (-74.00721 40.74697)
3,3,POINT (-74.00722 40.74722)
4,4,POINT (-74.00728 40.74667)
...,...,...
2295,2295,POINT (-73.97976 40.71559)
2296,2296,POINT (-73.97987 40.71361)
2297,2297,POINT (-73.98387 40.72077)
2298,2298,POINT (-73.97780 40.71833)


In [ ]:
df_road = gpd.read_file('/content/road_locations.shp')
# df_road.drop('road_id', axis=1, inplace=True)
df_road

,id,Road_Name,geometry
0,1,Front St,"LINESTRING (-74.01301 40.70215, -74.01167 40.7..."
1,1,Front St,"LINESTRING (-74.01171 40.70247, -74.00999 40.7..."
2,1,Front St,"LINESTRING (-74.00577 40.70566, -74.00396 40.7..."
3,1,Front St,"LINESTRING (-74.00385 40.70651, -74.00142 40.7..."
4,2,Water,"LINESTRING (-74.01270 40.70275, -74.01180 40.7..."
...,...,...,...
331,248,Kenmare St,"LINESTRING (-73.99741 40.72164, -73.99389 40.7..."
332,249,Duane St,"LINESTRING (-74.00203 40.71159, -74.00252 40.7..."
333,250,City Hall,"LINESTRING (-74.00296 40.71334, -74.00150 40.7..."
334,251,Dutch,"LINESTRING (-74.00799 40.70899, -74.00720 40.7..."


In [ ]:
def find_closest_road(point, roads_df, min_distance):
    closest_road_id = []

    for idx, road in roads_df.iterrows():
        distance = point.distance(road['geometry'])
        if distance < min_distance:
            closest_road_id.append(road['id'])

    return closest_road_id

In [ ]:
df_point['closest_road_id'] = df_point.apply(lambda row: find_closest_road(row['geometry'], df_road, 0.0003), axis=1)
# df_point

In [ ]:
def blockInfo(shapefile):
    gdf = gpd.read_file(shapefile)
    block_info_list = []
    for idx, row in gdf.iterrows():
        block_id = row['id']
        polygon = row['geometry']
        corners = list(polygon.exterior.coords)
        block_info = {
            'blockID': block_id,
            'corners': corners
        }
        block_info_list.append(block_info)
    return block_info_list

In [ ]:
def blockIdentify(hydrantCenter, blocks):
    hydrantPoint = Point(hydrantCenter)
    for block in blocks:
        blockCoords = block['corners']
        blockPolygon = Polygon(blockCoords)
        if hydrantPoint.within(blockPolygon):
            hydrantBlockID = block['blockID']
            return hydrantBlockID
    return None

In [ ]:
blocks = blockInfo("/content/1920s_blocks_complete.shp")
df_point['block_id'] = df_point.apply(lambda row: blockIdentify(row['geometry'], blocks), axis=1)

In [ ]:
# Check if the entire dataframe contains NA values
is_na_anywhere = df_point.isna().any().any()

print("Dataframe contains NA values:", is_na_anywhere)

Dataframe contains NA values: True


In [ ]:
df_point

,id,geometry,closest_road_id,block_id
0,0,POINT (-74.00698 40.74748),[119],1.0
1,1,POINT (-74.00722 40.74722),"[119, 170]",2.0
2,2,POINT (-74.00721 40.74697),[120],2.0
3,3,POINT (-74.00722 40.74722),"[119, 170]",2.0
4,4,POINT (-74.00728 40.74667),[120],3.0
...,...,...,...,...
2295,2295,POINT (-73.97976 40.71559),[234],505.0
2296,2296,POINT (-73.97987 40.71361),"[30, 35, 235]",901.0
2297,2297,POINT (-73.98387 40.72077),[221],437.0
2298,2298,POINT (-73.97780 40.71833),"[199, 235]",444.0


In [ ]:
class Hydrant:

    def __init__(self, id, x, y, closest_road_id): # constructor

        # From shapefile
        self.pointID = id
        self.x = x
        self.y = y
        self.blockID = 0
        self.roadID = closest_road_id

        # Will initialize once into network generation
        self.branchID = 0
        self.neighbor = [] # List to store all the neighbors of this hydrant
        self.edge = {} # Dict to store edge information

In [ ]:
# Create an empty list to store the 'hydrant' instances
hydrant_instances = []

# Iterate through each row in the DataFrame and create 'hydrant' instances
for index, row in df_point.iterrows():
    # Get the Point object from the 'geometry' column
    point = row['geometry']

    # Extract 'x' and 'y' coordinates from the Point object
    x, y = point.x, point.y

    # Create 'hydrant' instance and add it to the list
    hydrant_obj = Hydrant(row['id'], x, y, row['closest_road_id'])
    hydrant_instances.append(hydrant_obj)

In [ ]:
# Function to calculate the Euclidean distance between two points
def distance(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

In [ ]:
# Threshold for maximum allowed edge length
max_edge_length = 0.002

In [ ]:
filtered_edges_cp = set()

# Extract x and y coordinates from hydrant instances
hydrant_points = [(hydrant.x, hydrant.y) for hydrant in hydrant_instances]

# Convert the list of points to a numpy array
points = np.array(hydrant_points)

# Perform Delaunay triangulation
triangulation = Delaunay(points)
len(triangulation.simplices)


# Loop through each triangle (simplex)
for simplex in triangulation.simplices:
    # Get the indices of the points forming the edges of the triangle
    #####print(simplex[0], simplex[1], simplex[2])
    edge1 = frozenset([simplex[0], simplex[1]])
    edge2 = frozenset([simplex[1], simplex[2]])
    edge3 = frozenset([simplex[0], simplex[2]])

    # Get the actual points from the 'points' list using the indices
    point1, point2, point3 = points[simplex[0]], points[simplex[1]], points[simplex[2]]
    #####print(point1, point2, point3)
    # Check if any edge length exceeds the maximum allowed length
    # If any edge length exceeds the limit, remove the corresponding edges from the set
    if distance(point1, point2) <= max_edge_length:
        filtered_edges_cp.add(edge1)
        # edge_obj = Edge(edge_id, simplex[0], simplex[1], distance(point1, point2))
        # edge_set.append(edge_obj)
        # edge_id += 1
    if distance(point2, point3) <= max_edge_length:
        filtered_edges_cp.add(edge2)
        # edge_obj = Edge(edge_id, simplex[1], simplex[2], distance(point2, point3))
        # edge_set.append(edge_obj)
        # edge_id += 1
    if distance(point1, point3) <= max_edge_length:
        filtered_edges_cp.add(edge3)
        # edge_obj = Edge(edge_id, simplex[0], simplex[2], distance(point1, point3))
        # edge_set.append(edge_obj)
        # edge_id += 1


In [ ]:
all_edges = []
for edge in filtered_edges_cp:
    edge_points = list(edge)
    # print("X Coordinate:", np.array(points)[edge_points, 0], "Y Coordinate:", np.array(points)[edge_points, 1])
    first_vertex = (np.array(points)[edge_points, 0][0], np.array(points)[edge_points, 1][0])
    second_vertex = (np.array(points)[edge_points, 0][1], np.array(points)[edge_points, 1][1])
    edge_endpoint = (first_vertex, second_vertex)
    # print("edge_endpoint", edge_endpoint)
    all_edges.append(tuple(edge_endpoint))

In [ ]:
import math
from sympy import Point, Line, pi


def dot(vA, vB):
    return vA[0]*vB[0]+vA[1]*vB[1]

def ang(lineA, lineB):
    # Get nicer vector form
    vA = [(lineA[0][0]-lineA[1][0]), (lineA[0][1]-lineA[1][1])]
    vB = [(lineB[0][0]-lineB[1][0]), (lineB[0][1]-lineB[1][1])]
    # Get dot prod
    dot_prod = dot(vA, vB)
    # Get magnitudes
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    # Get cosine value
    cos_ = dot_prod/magA/magB
    # Get angle in radians and then convert to degrees
    angle = math.acos(dot_prod/magB/magA)
    # Basically doing angle <- angle mod 360
    ang_deg = math.degrees(angle)%360

    if ang_deg-180>=0:
        # As in if statement
        return 360 - ang_deg
    else:
        return ang_deg


def angle_cost_function(edge, all_edges):
  obtuse_angles = 0
  acute_angles = 0
  neighboring_edges_adjacency_list = find_neighbors(edge, all_edges)
  cost = 0
  for neighbor in neighboring_edges_adjacency_list[edge]:
    try:
      angle = (ang(neighbor, edge))
      # print("angle:", angle)
      if (angle > 150):
        cost += (0.2)
        obtuse_angles += 1
      else:
        cost += (0.8)
        acute_angles += 1
    except ValueError:
        # print("ValueError")
        pass
  # print("obtuse angles:", obtuse_angles)
  # print("acute angles:", acute_angles)
  return cost


#   # get all of the edge's neighbors
#   for edge in filtered_edges_cp:
#     edge_points = list(edge)
#     # print("X Coordinate:", np.array(points)[edge_points, 0], "Y Coordinate:", np.array(points)[edge_points, 1])
#     first_vertex = (np.array(points)[edge_points, 0][0], np.array(points)[edge_points, 1][0])
#     second_vertex = (np.array(points)[edge_points, 0][1], np.array(points)[edge_points, 1][1])
#     edge_endpoint = (first_vertex, second_vertex)
#     # print("edge_endpoint", edge_endpoint)
#     all_edges.append(tuple(edge_endpoint))

# print("len(all_edges)", len(all_edges))

# count = 0
def find_neighbors(edge, all_edges):
  neighboring_edges_adjacency_list = {}
  target_coordinate = edge[0]
  for search_edge1 in all_edges:
    if search_edge1[0] == target_coordinate or search_edge1[1] == target_coordinate:
      if not(edge in neighboring_edges_adjacency_list):
          # print("new target key created")
          neighboring_edges_adjacency_list[edge] = []
      # print("new value added to target")
      neighboring_edges_adjacency_list[edge].append(search_edge1)
      # print("after adding the value:", neighboring_edges_adjacency_list[target])
  target_coordinate = edge[1]
  for search_edge2 in all_edges:
    if search_edge2[0] == target_coordinate or search_edge2[1] == target_coordinate:
      if not(edge in neighboring_edges_adjacency_list):
          # print("new target key created")
          neighboring_edges_adjacency_list[edge] = []
      # print("new value added to target")
      neighboring_edges_adjacency_list[edge].append(search_edge2)
      # print("after adding the value:", neighboring_edges_adjacency_list[target])
  return neighboring_edges_adjacency_list

In [ ]:
for edge in all_edges:
  print(angle_cost_function(edge, all_edges))



Streaming output truncated to the last 5000 lines.
9.8
8.4
10.000000000000002
9.8
12.200000000000001
8.4
9.200000000000001
8.6
9.6
7.6
8.2
9.200000000000001
8.799999999999999
8.799999999999999
7.8
10.600000000000001
8.399999999999999
10.800000000000002
6.0
9.0
9.8
8.799999999999999
9.8
9.0
6.6
8.799999999999999
10.600000000000001
11.200000000000001
7.3999999999999995
8.2
10.800000000000002
7.8
6.6
8.6
8.799999999999999
9.2
9.8
9.6
9.0
6.8
10.4
6.6
7.199999999999999
9.8
10.8
7.6
5.2
9.0
6.2
8.799999999999999
10.600000000000001
7.6
9.2
10.6
9.0
7.6
11.200000000000001
6.8
6.199999999999999
10.4
8.999999999999998
9.200000000000001
4.4
8.2
9.0
9.0
8.4
8.2
9.6
5.4
9.0
10.6
7.3999999999999995
6.6
9.200000000000001
8.2
7.3999999999999995
6.8
6.8
8.4
7.199999999999999
9.0
9.0
8.2
6.8
8.6
9.6
8.2
9.8
6.8
7.6
9.0
6.0
8.799999999999999
8.2
10.600000000000001
6.6
9.8
10.0
9.0
7.6
9.200000000000001
9.8
9.6
7.199999999999999
9.6
9.200000000000001
10.8
6.6
12.000000000000002
8.799999999999999
9.0
9.8
